In [1]:
import requests
import pandas as pd

class ScriptData:
    def __init__(self, api_key):
        self.api_key = api_key
        self.data = {}

    def fetch_intraday_data(self, script):
        url = f"https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={script}&interval=1min&apikey={self.api_key}"
        response = requests.get(url)
        data = response.json()
        self.data[script] = data['Time Series (1min)']
    
    def convert_intraday_data(self, script):
        raw_data = self.data[script]
        timestamp = [pd.Timestamp(time) for time in raw_data.keys()]
        open_price = [float(raw_data[time]['1. open']) for time in raw_data.keys()]
        high = [float(raw_data[time]['2. high']) for time in raw_data.keys()]
        low = [float(raw_data[time]['3. low']) for time in raw_data.keys()]
        close = [float(raw_data[time]['4. close']) for time in raw_data.keys()]
        volume = [int(raw_data[time]['5. volume']) for time in raw_data.keys()]
        self.data[script] = pd.DataFrame({'timestamp': timestamp,
                                          'open': open_price,
                                          'high': high,
                                          'low': low,
                                          'close': close,
                                          'volume': volume})
    
    def __getitem__(self, script):
        return self.data[script]
    
    def __setitem__(self, script, value):
        self.data[script] = value
    
    def __contains__(self, script):
        return script in self.data

def indicator1(df, timeperiod):
    indicator = df['close'].rolling(window=timeperiod).mean()
    return pd.DataFrame({'timestamp': df['timestamp'], 'indicator': indicator})


In [2]:
# Initialize ScriptData class
api_key = "CKQP8IMGJH343SBG"
script_data = ScriptData(api_key)

# Fetch intraday data for Google
script_data.fetch_intraday_data("GOOGL")

# Convert fetched data to pandas DataFrame
script_data.convert_intraday_data("GOOGL")
df = script_data["GOOGL"]




In [3]:
print(df)

             timestamp     open     high      low    close  volume
0  2023-03-02 20:00:00  91.8500  91.8500  91.8500  91.8500     350
1  2023-03-02 19:57:00  91.7500  91.7500  91.7500  91.7500     109
2  2023-03-02 19:55:00  91.7500  91.7500  91.7500  91.7500     165
3  2023-03-02 19:53:00  91.8500  91.8500  91.8500  91.8500     100
4  2023-03-02 19:44:00  91.7900  91.7900  91.7900  91.7900     201
..                 ...      ...      ...      ...      ...     ...
95 2023-03-02 16:17:00  91.9000  91.9000  91.9000  91.9000    1192
96 2023-03-02 16:16:00  91.9600  91.9600  91.9000  91.9000    2048
97 2023-03-02 16:15:00  91.9600  92.0000  91.9600  92.0000   49991
98 2023-03-02 16:14:00  91.9000  91.9300  91.9000  91.9300     553
99 2023-03-02 16:13:00  91.9567  91.9567  91.9567  91.9567     609

[100 rows x 6 columns]


In [4]:
if "AAPL" in script_data:
    print("AAPL data found.")

In [5]:
# Calculate moving average indicator
indicator_df = indicator1(df, 5)

In [6]:
print(indicator_df)

             timestamp  indicator
0  2023-03-02 20:00:00        NaN
1  2023-03-02 19:57:00        NaN
2  2023-03-02 19:55:00        NaN
3  2023-03-02 19:53:00        NaN
4  2023-03-02 19:44:00   91.79800
..                 ...        ...
95 2023-03-02 16:17:00   91.89228
96 2023-03-02 16:16:00   91.89428
97 2023-03-02 16:15:00   91.91628
98 2023-03-02 16:14:00   91.92628
99 2023-03-02 16:13:00   91.93734

[100 rows x 2 columns]


In [21]:
class Strategy:
    def __init__(self, api_key):
        self.api_key = api_key
    
    def generate_signals(self, script):
        script_data = ScriptData(api_key=self.api_key)
        script_data.fetch_intraday_data(script)
        script_data.convert_intraday_data(script)
        df = script_data[script]
        indicator_data = indicator1(df, timeperiod=20)['indicator']
        close_data = df['close']
        signals = pd.DataFrame({'timestamp': df['timestamp']})
        signals.loc[indicator_data > close_data, 'signal'] = 'BUY'
        signals.loc[indicator_data < close_data, 'signal'] = 'SELL'
        signals.loc[indicator_data == close_data, 'signal'] = 'NO_SIGNAL'
        return signals


In [26]:
my_strategy = Strategy(api_key= "CKQP8IMGJH343SBG")
script = 'NVDA'

signals = my_strategy.generate_signals(script)
buy_sell_signals = signals.loc[(signals['signal'] == 'BUY') | (signals['signal'] == 'SELL')]

print(buy_sell_signals)

             timestamp signal
19 2023-03-02 19:34:00    BUY
20 2023-03-02 19:33:00    BUY
21 2023-03-02 19:32:00    BUY
22 2023-03-02 19:31:00    BUY
23 2023-03-02 19:30:00    BUY
..                 ...    ...
95 2023-03-02 17:32:00   SELL
96 2023-03-02 17:31:00   SELL
97 2023-03-02 17:30:00   SELL
98 2023-03-02 17:29:00   SELL
99 2023-03-02 17:27:00   SELL

[81 rows x 2 columns]
